## **Environment Setup**

In [1]:
import os 
os.chdir ('../src')

## **Importing Libraries**

In [12]:
from init import pd, nlp, tqdm
from entity import get_entities, get_relation

In [3]:
pd.set_option('display.max_colwidth', 200)

In [4]:
ROOT_PATH = '/run/media/sri/OS/Users/sriva/Desktop/Sem 5/'
ROOT_PATH += 'Knowledge Representation/Project/'
ROOT_PATH += 'Complex-Question-Answering-using-Knowledge-Graphs/'

## **Reading the data**

In [5]:
candidate_sentences = pd.read_csv(ROOT_PATH + "data/wiki_sentences_v2.csv")

## **Getting Entity Pairs**

In [6]:
entity_pairs = []

In [7]:
for i in tqdm(candidate_sentences["sentence"]):
    entity_pairs.append(get_entities(i))

100%|██████████| 4318/4318 [00:47<00:00, 90.01it/s]


In [8]:
for i in range(10):
    print(entity_pairs[i])

['connie', 'own']
['later  scream', 'distance']
['christian', 'then  elder']
['temple', 'fire']
['', 'outside cult him']
['it', 'religious  awakening']
['c. mackenzie', 'craig cast']
['later craig di francia', 'action cast']
['sebastian  maniscalco', 'later paul ben cast']
['we', 'just  film']


In [13]:
relations = [get_relation(i) for i in tqdm(candidate_sentences['sentence'])]

100%|██████████| 4318/4318 [00:44<00:00, 96.06it/s]


In [21]:
pd.Series(relations).value_counts()[:50]

is               359
was              308
released on       89
are               72
include           71
were              62
released          40
composed by       35
became            34
's                32
have              31
has               30
become            28
released in       27
included          24
had               21
produced          21
called            19
been              19
considered        18
made              18
used              17
scheduled         17
be                16
directed by       16
hired             14
written by        14
wrote             14
introduced in     13
produced by       13
received          13
went              13
wanted            12
sold              12
began in          11
won               11
set               11
began             11
cast as           11
gave              10
includes          10
stars             10
reported           9
shot in            9
going              9
directed           9
produced in        9
opened       

In [11]:
from init import Matcher

for i in tqdm(candidate_sentences['sentence']):

    doc = nlp(i)

    matcher = Matcher(nlp.vocab)

    pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

    matcher.add("matching_1", [pattern]) 

    matches = matcher(doc)
    k = len(matches) - 1

    if k == -1:
        continue
    else:
        span = doc[matches[k][1]:matches[k][2]] 

100%|██████████| 4318/4318 [00:41<00:00, 102.88it/s]
